In [1]:
# Author: Alexander Maksiaev
# Purpose: Create the dictionary used for sentiment analysis

In [2]:
# Housekeeping: Importing libraries, switching directories, etc.

import os
import pandas as pd
import csv 
import numpy as np

os.getcwd()

results_dir = r"C:\Users\maksi\Documents\UVA\Research\DMP\Results"

results = os.listdir(results_dir)

In [3]:
# Getting the big data frame

results_clean = []
df = pd.read_csv("Sentiment_Analysis_Results_2.csv", low_memory=False)
print(df)
print(len(df))
original_df = df

                                             StartDate  \
0                                           Start Date   
1    {"ImportId":"startDate","timeZone":"America/De...   
2                                  2023-11-05 14:39:31   
3                                  2023-11-06 14:32:51   
4                                  2023-11-06 16:33:32   
..                                                 ...   
529                                2023-11-19 17:29:19   
530                                2023-11-17 12:56:00   
531                                2023-11-18 21:07:12   
532                                2023-11-19 20:00:38   
533                                2023-11-19 20:42:51   

                                               EndDate                 Status  \
0                                             End Date          Response Type   
1    {"ImportId":"endDate","timeZone":"America/Denv...  {"ImportId":"status"}   
2                                  2023-11-05 14:46:00      

In [4]:
# Cleaning 

# Study agreement columns: Q11047, Q11050. Only include people who agreed to the study

df = df[df['Q11047'] == "1"]
df = df[df['Q11050'] == "1"]
words = original_df.iloc[[0]] # Row that has words to form the dictionary
print(words)
including_words = [words, df]
df = pd.concat(including_words, axis=0) # Add the words 
print(df)
print(len(df))

    StartDate   EndDate         Status   IPAddress  Progress  \
0  Start Date  End Date  Response Type  IP Address  Progress   

   Duration (in seconds)  Finished   RecordedDate   ResponseId  \
0  Duration (in seconds)  Finished  Recorded Date  Response ID   

     RecipientLastName  ... QID10831     QID10832   QID10833  \
0  Recipient Last Name  ...  weights  impairments  terrorist   

                         QID10834  \
0  Please choose the option [+3].   

                                              Q10051  \
0  Below are some questions for demographic purpo...   

                                    Q11051              Q11052  \
0  How old are you? Please enter a number.  What year are you?   

                                             Q11053  \
0  Are you a Psychology or Cognitive Science major?   

                                              Q11054  \
0  Have you learned about autism in any UVa cours...   

                                              Q11049  
0  Debrie

In [5]:
# Split the data frame into mini dataframes representing each survey

# 1: QID8457 -- QID8664
beginning = df.loc[:, "StartDate":"QID12"] # Study agreement questions and "golden item" questions
# print(beginning)
end = df.loc[:, "Q10051":] # Demographics
# print(end)
# example = (df.loc[:,"QID8456":"QID8664"]) # 209 columns of words and valences
# print(example)
# 2: QID1142 -- QID1349
# 3: (etc.)

# df_example = [beginning, example, end]
# df_example = pd.concat(df_example, axis=1) 
# print(df_example)

# Create a dictionary holding all the surveys
dict_of_dfs = {}
# dict_of_questions = {}
questions = df.loc[:,"QID8456":"QID10834"]
# questions = df.loc[:, "QID2":"QID10834"]
for i in range(48):
    name_of_df = "df_" + str(i)
    questions_specific = questions.iloc[:, (i*209):((i+1)*209)] # Get only the questions of that survey
    
    
    df_entry = [beginning, questions_specific, end] # But include the study agreements and demographics
    df_entry = pd.concat(df_entry, axis=1)
    df_entry = df_entry.dropna(thresh=209,axis=0) # Only include people who filled out that particular survey
    # Make the first row the columns
    df_entry.columns = df_entry.iloc[0]
    df_entry = df_entry.apply(pd.to_numeric, errors='coerce', downcast='integer')
    
    # Get the questions only
#     practice_questions = df_entry.iloc[:, "QID3":"QID12"]
#     questions_full = [practice_questions, questions_specific]
#     questions_full = pd.concat(questions_full, axis=1)
    
    
    dict_of_dfs[name_of_df] = df_entry # Add to dictionary
#     dict_of_questions[name_of_df] = questions_full

# Example: Survey #8 (ninth survey)
dict_of_dfs["df_8"]
# dict_of_questions["df_8"]

,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,canning,aims,terrorist,Please choose the option [+3].,Below are some questions for demographic purposes.,How old are you? Please enter a number.,What year are you?,Are you a Psychology or Cognitive Science major?,Have you learned about autism in any UVa courses presently or in the past?,"Debriefing Form\nProtocol #: 6251: Sentiment Analysis \nThank you for participating in this study! In this study, you rated several words for how\nnegative or positive they were. We will calculate, for each word, the average rating and\nuse those averages to be able to objectively quantify the ""sentiment"" of a piece of text.\nOur particular interest is in assessing the average valence of the words used in sections of\npopular college-level textbooks that describe autism. We will investigate how the valence\nhas changed over time, whether textbooks in some sub-fields use more positive or\nnegative words than others, and so on. This will allow us to understand and document\nhow attitudes about autism are (or are not) changing in college-level textbooks. Sentiment analysis has been used for a number of purposes, including customer reviews\nof products or services, social media analysis, and so on. It has also been used in the\nhumanities to try to quantify the emotions or sentiments in stories and poems. \nIf you feel especially concerned about any aspects of this study, please feel free to contact\nProf. Jaswal at jaswal@virginia.edu about options for counseling. Alternatively, you\ncould also phone the UVA Counseling and Psychological Services (434-243-5556) or the\nMary D. Ainsworth Psychological Clinic in the psychology department (434-982-4737). \nThank you for your participation in this study. \nPlease contact the researchers on the study team listed below to: ● Obtain more information or if you have further questions about the study. ● Report an illness, injury, or other problem. ● Leave the study before it is finished. Vikram Jaswal, Ph.D.\nProfessor of Psychology\nDept. of Psychology PO Box 400400\nUniversity of Virginia\nCharlottesville, VA 22904-4400 Telephone: (434) 982-4709 Email: jaswal@virginia.edu \nYou may also report a concern about a study or ask questions about your rights as a\nresearch subject by contacting the Institutional Review Board listed below. Tonya R. Moon, Ph.D.\nChair, Institutional Review Board for the Social and Behavioral Sciences One Morton Dr Suite 400\nUniversity of Virginia, P.O. Box 800392\nCharlottesville, VA 22908-0392 Telephone: (434) 924-5999 Email: irbsbshelp@virginia.edu Website: https://research.virginia.edu/irb-sbs Website for Research Participants: https://research.virginia.edu/research-participants \nUVA IRB-SBS # 6251 \nAdditional Reading: To learn more general information about sentiment analysis, you can visit:\nhttps://aws.amazon.com/what-is/sentiment-analysis/ Wankhade, M., Rao, A.C.S. & Kulkarni, C. (2022). A survey on sentiment analysis\nmethods, applications, and challenges. Artificial Intelligence Review, 55, 5731–5780\n(2022). https://doi.org/10.1007/s10462-022-10144-1"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,0.0,NaN,100.0,1082.0,1.0,NaN,NaN,NaN,...,0.0,0.0,-4.0,3.0,NaN,18.0,1.0,2.0,2.0,NaN
75,NaN,NaN,0.0,NaN,100.0,1328.0,1.0,NaN,NaN,NaN,...,0.0,1.0,-4.0,3.0,NaN,19.0,2.0,2.0,2.0,NaN
109,NaN,NaN,0.0,NaN,100.0,604.0,1.0,NaN,NaN,NaN,...,0.0,0.0,-4.0,3.0,NaN,19.0,2.0,2.0,2.0,NaN
144,NaN,NaN,0.0,NaN,100.0,833.0,1.0,NaN,NaN,NaN,...,0.0,1.0,-4.0,3.0,NaN,18.0,1.0,2.0,2.0,NaN
208,NaN,NaN,0.0,NaN,100.0,1022.0,1.0,NaN,NaN,NaN,...,0.0,-1.0,-4.0,3.0,NaN,19.0,1.0,2.0,2.0,NaN
247,NaN,NaN,0.0,NaN,100.0,441.0,1.0,NaN,NaN,NaN,...,0.0,1.0,-3.0,3.0,NaN,18.0,1.0,2.0,2.0,NaN
301,NaN,NaN,0.0,NaN,100.0,752.0,1.0,NaN,NaN,NaN,...,-1.0,0.0,-4.0,3.0,NaN,19.0,2.0,2.0,1.0,NaN
336,NaN,NaN,0.0,NaN,100.0,873.0,1.0,NaN,NaN,NaN,.

In [6]:
# Exclude people who did not answer "Please choose option [x]" questions correctly

# In order:
# "Please choose the option [+2]."
# "Please choose the option [-1]."
# "Please choose the option [0]."
# "Please choose the option [+3]."
non_excluded = []
for key in dict_of_dfs.keys():
#     print("Change below")
    dataframe = dict_of_dfs[key]
#     print(len(dataframe))
    dataframe = dataframe.loc[(dataframe["Please choose the option [+2]."] == 2) & (dataframe["Please choose the option [-1]."] == -1) & (dataframe["Please choose the option [0]."] == 0) & (dataframe["Please choose the option [+3]."] == 3)]
    dict_of_dfs[key] = dataframe
    non_excluded.append(len(dataframe))
#     print("------")

print(non_excluded)

[11, 9, 11, 10, 9, 11, 10, 11, 11, 11, 9, 9, 11, 10, 10, 11, 10, 11, 11, 10, 9, 11, 11, 10, 10, 10, 11, 9, 10, 10, 11, 10, 10, 11, 9, 11, 11, 10, 12, 12, 10, 12, 11, 11, 11, 11, 10, 11]


In [7]:
# Keep participants who got > 80% golden items within +/- 2

how_many_people_correct = []
eighty_correct_2 = []
for key in dict_of_dfs.keys():
#     print("Change below")
    dataframe = dict_of_dfs[key]
#     print(len(dataframe))

    dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
    counter_people = False
    for index, row in dataframe.iterrows(): # Row is the particpant
        # Give a point if they get the ideal solution
        counter = 0
        if row["love"] >= 1:
            counter += 1
        if row["charming"] >= 1:
            counter += 1
        if row["proud"] >= 0:
            counter += 1
        if row["safe"] >= 0:
            counter += 1
        if row["resolvable"] >= -1 and row["resolvable"] <= 3:
            counter += 1
        if row["prepared"] >= -1 and row["prepared"] <= 3:
            counter += 1
        if row["reach"] >= -2 and row["reach"] <= 2:
            counter += 1
        if row["backed"] >= -2 and row["backed"] <= 2:
            counter += 1
        if row["shy"] >= -3 and row["shy"] <= 1:
            counter += 1
        if row["stutter"] >= -3 and row["stutter"] <= 1:
            counter += 1
        if row["depriving"] <= 0:
            counter += 1
        if row["ungrateful"] <= 0:
            counter += 1
        if row["cruelness"] <= -1:
            counter += 1
        if row["repulse"] <= -1:
            counter += 1
        if row["slavery"] <= -2:
            counter += 1
        if row["terrorist"] <= -2:
            counter += 1
#         print(counter)
        if counter / 16 >= 0.8: # Keep the person if they got >80% correct +/- 2
            counter_people = True
    
        if not counter_people: # If they didn't do well, remove them
            dataframe.drop([index])
        
        
            
    
#     dataframe = dataframe[dataframe["love"] >= 1] # 3
#     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
#     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
#     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
#     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
#     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
#     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
#     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
#     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
#     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
#     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
#     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
#     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
#     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
#     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
#     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
    dict_of_dfs[key] = dataframe
#     print("counter_people: ", counter_people)
#     how_many_people_correct.append(counter_people)
#     eighty_correct_2.append(counter_people)
# print(eighty_correct_2)
# print(sum(how_many_people_correct) / len(how_many_people_correct))
# print(min(how_many_people_correct))

print(dict_of_dfs["df_27"])

0    Start Date  End Date  Response Type  IP Address  Progress  \
15          NaN       NaN            0.0         NaN     100.0   
69          NaN       NaN            0.0         NaN     100.0   
159         NaN       NaN            0.0         NaN     100.0   
188         NaN       NaN            0.0         NaN     100.0   
259         NaN       NaN            0.0         NaN     100.0   
289         NaN       NaN            0.0         NaN     100.0   
416         NaN       NaN            0.0         NaN     100.0   
440         NaN       NaN            0.0         NaN     100.0   
475         NaN       NaN            0.0         NaN     100.0   

0    Duration (in seconds)  Finished  Recorded Date  Response ID  \
15                   478.0       1.0            NaN          NaN   
69                   643.0       1.0            NaN          NaN   
159                  468.0       1.0            NaN          NaN   
188                 3127.0       1.0            NaN          NaN   

In [8]:
# Format dataframes to get only words for dictionary

dictionary = pd.DataFrame()

for key in dict_of_dfs.keys():

    dataframe = dict_of_dfs[key]
    
    # Get the questions only
    
    dataframe = dataframe.loc[:, "stutter":"Below are some questions for demographic purposes."]
    
    dataframe = dataframe.loc[:,~dataframe.columns.str.endswith('.')]

    dict_of_dfs[key] = dataframe
    
    

print(dict_of_dfs["df_32"])

    

0    stutter  depriving  charming  backed  resolvable  proud  shy  repulse  \
58      -3.0       -2.0       3.0     0.0         2.0    4.0 -1.0     -3.0   
113     -1.0       -3.0       3.0     2.0         1.0    1.0 -2.0     -4.0   
174     -2.0       -3.0       3.0    -1.0         1.0    3.0 -1.0     -4.0   
215     -2.0       -4.0       3.0     0.0         2.0    3.0 -1.0     -3.0   
256     -1.0       -3.0       3.0     0.0         0.0    3.0  0.0     -3.0   
282     -2.0       -3.0       3.0    -2.0         1.0    3.0 -1.0     -3.0   
347     -1.0       -2.0       4.0     0.0         2.0    4.0  0.0      0.0   
377     -1.0       -1.0       2.0    -1.0         1.0    3.0 -1.0     -3.0   
459     -3.0       -1.0       3.0     2.0         1.0    4.0 -1.0     -4.0   
473      2.0       -1.0       3.0     0.0         4.0    4.0  0.0      0.0   

0    ungrateful  slavery  ...  venter  army  anticipatory  writer  jointly  \
58         -3.0     -4.0  ...     0.0   1.0           1.0     1

In [9]:
# Create the dictionary (in dataframe form)
# Format: word (tab) mean (tab) standard deviation (tab) raw scores (as a bracketed list)

all_words = []
means = []
stdevs = []
raw_scores = []
for key in dict_of_dfs.keys():

    dataframe = dict_of_dfs[key]
    
    # Make a column for the words
    for word in dataframe.columns:
        if word not in all_words: # Only include the "golden items" for the first survey, to avoid too much data for those words
            all_words.append(word)
            scores = dataframe[word].dropna().to_list() # Get the scores as a list
            means.append(np.mean(scores)) # Get the means
            stdevs.append(np.std(scores)) # Get the standard deviations
            raw_scores.append(scores) # Get the scores as a list but put them into a bigger list
            
        
# Create the dataframe            
dictionary["word"] = all_words
dictionary["mean"] = means
dictionary["std_dev"] = stdevs
dictionary["raw_scores"] = raw_scores

dictionary



,word,mean,std_dev,raw_scores
0,stutter,-1.727273,0.862439,"[-2.0, 0.0, -3.0, -2.0, -1.0, -1.0, -2.0, -3.0..."
1,depriving,-2.545455,1.075651,"[-3.0, 0.0, -4.0, -3.0, -3.0, -3.0, -3.0, -3.0..."
2,charming,2.636364,0.481046,"[3.0, 2.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 2.0, ..."
3,backed,0.545455,0.987525,"[2.0, 0.0, 0.0, 0.0, 2.0, 2.0, -1.0, 0.0, 0.0,..."
4,resolvable,1.181818,1.336085,"[-1.0, 0.0, 3.0, 3.0, 2.0, 2.0, 1.0, 1.0, -1.0..."
...,...,...,...,...
9508,develop,1.363636,0.771389,"[2.0, 1.0, 0.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, ..."
9509,learn,2.181818,1.113404,"[3.0, 1.0, 0.0, 3.0, 3.0, 2.0, 2.0, 2.0, 4.0, ..."
9510,resiliency,2.818182,1.266217,"[4.0, 2.0, 0.0, 3.0, 3.0, 3.0, 4.0, 3.0, 4.0, ..."
9511,weights,-0.181818,0.935966,"[0.0, 0.0, 0.0, 0.0, -2.0, 0.0, 0.0, 1.0, -2.0..."


In [10]:
dict_dir = r"C:\Users\maksi\Documents\UVA\Research\DMP\Dictionaries"

os.chdir(dict_dir)
dictionary.to_csv('final_dictionary.txt', header=None, index=None, sep='\t')

In [11]:
# # Exclude people who did not get within +/- 0 of all of the "golden items" (Not many got all of them within +/- 1...)

# how_many_people_correct = []
# golden_items_correct = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
#     original_number_of_people = len(dataframe)
# #     print(len(dataframe))
#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
#     dataframe = dataframe[dataframe["love"] == 3] # 3
#     dataframe = dataframe.loc[(dataframe["charming"] == 3)] # 3
#     dataframe = dataframe.loc[(dataframe["proud"] == 2)] # 2
#     dataframe = dataframe.loc[(dataframe["safe"] == 2)] # 2
#     dataframe = dataframe.loc[(dataframe["resolvable"] == 1)] # 1
#     dataframe = dataframe.loc[(dataframe["prepared"] == 1)] # 1
#     dataframe = dataframe.loc[(dataframe["reach"] == 0)] # 0
#     dataframe = dataframe.loc[(dataframe["backed"] == 0)] # 0
#     dataframe = dataframe.loc[(dataframe["shy"] == -1)] # -1
#     dataframe = dataframe.loc[(dataframe["stutter"] == -1)] # -1
#     dataframe = dataframe.loc[(dataframe["depriving"] == -2)] # -2
#     dataframe = dataframe.loc[(dataframe["ungrateful"] == -2)] # -2
#     dataframe = dataframe.loc[(dataframe["cruelness"] == -3)] # -3
#     dataframe = dataframe.loc[(dataframe["repulse"] == -3)] # -3
#     dataframe = dataframe.loc[(dataframe["slavery"] == -4)] # -4
#     dataframe = dataframe.loc[(dataframe["terrorist"] == -4)] # -4

#     final_number_of_people = len(dataframe)
    
# #     dict_of_dfs[key] = dataframe
# #     print(len(dataframe))
#     how_many_people_correct.append(len(dataframe))
    
#     golden_items_correct.append(final_number_of_people/original_number_of_people)
    
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))
# print(golden_items_correct)

In [12]:
# # Exclude people who did not get within +/- 1 of all of the "golden items" (Not many got all of them within +/- 1...)

# how_many_people_correct = []
# golden_items_correct_1 = []

# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))
#     original_number_of_people = len(dataframe)

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
#     dataframe = dataframe[dataframe["love"] >= 2] # 3
#     dataframe = dataframe.loc[(dataframe["charming"] >= 2)] # 3
#     dataframe = dataframe.loc[(dataframe["proud"] >= 1) & (dataframe["proud"] <= 3)] # 2
#     dataframe = dataframe.loc[(dataframe["safe"] >= 1) & (dataframe["safe"] <= 3)] # 2
#     dataframe = dataframe.loc[(dataframe["resolvable"] >= 0) & (dataframe["resolvable"] <= 2)] # 1
#     dataframe = dataframe.loc[(dataframe["prepared"] >= 0) & (dataframe["prepared"] <= 2)] # 1
#     dataframe = dataframe.loc[(dataframe["reach"] >= -1) & (dataframe["reach"] <= 1)] # 0
#     dataframe = dataframe.loc[(dataframe["backed"] >= -1) & (dataframe["backed"] <= 1)] # 0
#     dataframe = dataframe.loc[(dataframe["shy"] >= -2) & (dataframe["shy"] <= 0)] # -1
#     dataframe = dataframe.loc[(dataframe["stutter"] >= -2) & (dataframe["stutter"] <= 0)] # -1
#     dataframe = dataframe.loc[(dataframe["depriving"] >= -3) & (dataframe["depriving"] <= -1)] # -2
#     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -3) & (dataframe["ungrateful"] <= -1)] # -2
#     dataframe = dataframe.loc[(dataframe["cruelness"] <= -2)] # -3
#     dataframe = dataframe.loc[(dataframe["repulse"] <= -2)] # -3
#     dataframe = dataframe.loc[(dataframe["slavery"] <= -3)] # -4
#     dataframe = dataframe.loc[(dataframe["terrorist"] <= -3)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print(len(dataframe))
#     how_many_people_correct.append(len(dataframe))
#     final_number_of_people = len(dataframe)

#     golden_items_correct_1.append(final_number_of_people/original_number_of_people)


# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))
# print(golden_items_correct_1)

In [13]:
# # Exclude people who did not get within +/- 2 of all of the "golden items" (Not many got all of them within +/- 1...)

# how_many_people_correct = []
# golden_items_correct_2 = []

# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))
#     original_number_of_people = len(dataframe)

# # titanic[titanic["Age"] > 35]
#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
#     dataframe = dataframe[dataframe["love"] >= 1] # 3
#     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
#     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
#     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
#     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
#     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
#     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
#     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
#     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
#     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
#     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
#     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
#     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
#     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
#     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
#     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print(len(dataframe))
#     how_many_people_correct.append(len(dataframe))
#     final_number_of_people = len(dataframe)

#     golden_items_correct_2.append(final_number_of_people/original_number_of_people)
    

# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))
# print(golden_items_correct_2)

In [14]:
# # How many golden items did people get correct on average?

# how_many_people_correct = []
# all_correct = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     participants = []
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] == 3:
#             counter += 1
#         if row["charming"] == 3:
#             counter += 1
#         if row["proud"] == 2:
#             counter += 1
#         if row["safe"] == 2:
#             counter += 1
#         if row["resolvable"] == 1:
#             counter += 1
#         if row["prepared"] == 1:
#             counter += 1
#         if row["reach"] == 0:
#             counter += 1
#         if row["backed"] == 0:
#             counter += 1
#         if row["shy"] == -1:
#             counter += 1
#         if row["stutter"] == -1:
#             counter += 1
#         if row["depriving"] == -2:
#             counter += 1
#         if row["ungrateful"] == -2:
#             counter += 1
#         if row["cruelness"] == -3:
#             counter += 1
#         if row["repulse"] == -3:
#             counter += 1
#         if row["slavery"] == -4:
#             counter += 1
#         if row["terrorist"] == -4:
#             counter += 1
# #         print(counter)
# #         if counter / 16 >= 0.8:
# #             counter_people += 1
    
#         correct_number = counter / 16
#         participants.append(correct_number)
        
#     all_correct.append(sum(participants)/len(participants))
    
# print(all_correct)

In [15]:
# # Ditto for only participants who got golden items within +/- 1

# how_many_people_correct = []
# all_correct_1 = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 2:
#             counter += 1
#         if row["charming"] >= 2:
#             counter += 1
#         if row["proud"] >= 1 and row["proud"] <= 3:
#             counter += 1
#         if row["safe"] >= 1 and row["safe"] <= 3:
#             counter += 1
#         if row["resolvable"] >= 0 and row["resolvable"] <= 2:
#             counter += 1
#         if row["prepared"] >= 0 and row["prepared"] <= 2:
#             counter += 1
#         if row["reach"] >= -1 and row["reach"] <= 1:
#             counter += 1
#         if row["backed"] >= -1 and row["backed"] <= 1:
#             counter += 1
#         if row["shy"] >= -2 and row["shy"] <= 0:
#             counter += 1
#         if row["stutter"] >= -2 and row["stutter"] <= 0:
#             counter += 1
#         if row["depriving"] <= -1 and row["depriving"] >= -3:
#             counter += 1
#         if row["ungrateful"] <= -1 and row["ungrateful"] >= -3:
#             counter += 1
#         if row["cruelness"] <= -2:
#             counter += 1
#         if row["repulse"] <= -2:
#             counter += 1
#         if row["slavery"] <= -3:
#             counter += 1
#         if row["terrorist"] <= -3:
#             counter += 1
# #         print(counter)
#         correct_number = counter / 16
#         participants.append(correct_number)
        
#     all_correct_1.append(sum(participants)/len(participants))
    
# print(all_correct_1)

In [16]:
# # Ditto for only participants who got golden items within +/- 2?

# how_many_people_correct = []
# all_correct_2 = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 1:
#             counter += 1
#         if row["charming"] >= 1:
#             counter += 1
#         if row["proud"] >= 0:
#             counter += 1
#         if row["safe"] >= 0:
#             counter += 1
#         if row["resolvable"] >= -1 and row["resolvable"] <= 3:
#             counter += 1
#         if row["prepared"] >= -1 and row["prepared"] <= 3:
#             counter += 1
#         if row["reach"] >= -2 and row["reach"] <= 2:
#             counter += 1
#         if row["backed"] >= -2 and row["backed"] <= 2:
#             counter += 1
#         if row["shy"] >= -3 and row["shy"] <= 1:
#             counter += 1
#         if row["stutter"] >= -3 and row["stutter"] <= 1:
#             counter += 1
#         if row["depriving"] <= 0:
#             counter += 1
#         if row["ungrateful"] <= 0:
#             counter += 1
#         if row["cruelness"] <= -1:
#             counter += 1
#         if row["repulse"] <= -1:
#             counter += 1
#         if row["slavery"] <= -2:
#             counter += 1
#         if row["terrorist"] <= -2:
#             counter += 1
# #         print(counter)
#         correct_number = counter / 16
#         participants.append(correct_number)
        
#     all_correct_2.append(sum(participants)/len(participants))
    
# print(all_correct_2)
    
            
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
# #     how_many_people_correct.append(counter_people)
    
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))

In [17]:
# # Exclude people who did not get 80% of the "golden items" 

# how_many_people_correct = []
# eighty_correct = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] == 3:
#             counter += 1
#         if row["charming"] == 3:
#             counter += 1
#         if row["proud"] == 2:
#             counter += 1
#         if row["safe"] == 2:
#             counter += 1
#         if row["resolvable"] == 1:
#             counter += 1
#         if row["prepared"] == 1:
#             counter += 1
#         if row["reach"] == 0:
#             counter += 1
#         if row["backed"] == 0:
#             counter += 1
#         if row["shy"] == -1:
#             counter += 1
#         if row["stutter"] == -1:
#             counter += 1
#         if row["depriving"] == -2:
#             counter += 1
#         if row["ungrateful"] == -2:
#             counter += 1
#         if row["cruelness"] == -3:
#             counter += 1
#         if row["repulse"] == -3:
#             counter += 1
#         if row["slavery"] == -4:
#             counter += 1
#         if row["terrorist"] == -4:
#             counter += 1
# #         print(counter)
#         if counter / 16 >= 0.8:
#             counter_people += 1
    
            
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
#     how_many_people_correct.append(counter_people)
#     eighty_correct.append(counter_people)
# print(eighty_correct)
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))

In [18]:
# # Ditto for only participants who got > 80% golden items within +/- 1

# how_many_people_correct = []
# eighty_correct_1 = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 2:
#             counter += 1
#         if row["charming"] >= 2:
#             counter += 1
# #         if row["proud"] >= 1 and row["proud"] <= 3:
# #             counter += 1
#         if row["safe"] >= 1 and row["safe"] <= 3:
#             counter += 1
#         if row["resolvable"] >= 0 and row["resolvable"] <= 2:
#             counter += 1
# #         if row["prepared"] >= 0 and row["prepared"] <= 2:
# #             counter += 1
#         if row["reach"] >= -1 and row["reach"] <= 1:
#             counter += 1
#         if row["backed"] >= -1 and row["backed"] <= 1:
#             counter += 1
#         if row["shy"] >= -2 and row["shy"] <= 0:
#             counter += 1
#         if row["stutter"] >= -2 and row["stutter"] <= 0:
#             counter += 1
#         if row["depriving"] <= -1 and row["depriving"] >= -3:
#             counter += 1
# #         if row["ungrateful"] <= -1 and row["ungrateful"] >= -3:
# #             counter += 1
#         if row["cruelness"] <= -2:
#             counter += 1
#         if row["repulse"] <= -2:
#             counter += 1
#         if row["slavery"] <= -3:
#             counter += 1
#         if row["terrorist"] <= -3:
#             counter += 1
# #         print(counter)
#         if counter / 13 >= 0.8:
#             counter_people += 1

#     how_many_people_correct.append(counter_people)
#     eighty_correct_1.append(counter_people)
# print(eighty_correct_1)
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))

In [19]:
# # Ditto for only participants who got > 80% golden items within +/- 2?

# how_many_people_correct = []
# eighty_correct_2 = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 1:
#             counter += 1
#         if row["charming"] >= 1:
#             counter += 1
#         if row["proud"] >= 0:
#             counter += 1
#         if row["safe"] >= 0:
#             counter += 1
#         if row["resolvable"] >= -1 and row["resolvable"] <= 3:
#             counter += 1
#         if row["prepared"] >= -1 and row["prepared"] <= 3:
#             counter += 1
#         if row["reach"] >= -2 and row["reach"] <= 2:
#             counter += 1
#         if row["backed"] >= -2 and row["backed"] <= 2:
#             counter += 1
#         if row["shy"] >= -3 and row["shy"] <= 1:
#             counter += 1
#         if row["stutter"] >= -3 and row["stutter"] <= 1:
#             counter += 1
#         if row["depriving"] <= 0:
#             counter += 1
#         if row["ungrateful"] <= 0:
#             counter += 1
#         if row["cruelness"] <= -1:
#             counter += 1
#         if row["repulse"] <= -1:
#             counter += 1
#         if row["slavery"] <= -2:
#             counter += 1
#         if row["terrorist"] <= -2:
#             counter += 1
# #         print(counter)
#         if counter / 16 >= 0.8:
#             counter_people += 1
    
            
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
#     how_many_people_correct.append(counter_people)
#     eighty_correct_2.append(counter_people)
# print(eighty_correct_2)
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))

In [20]:
# # How many golden items did people get correct valence on average?

# how_many_people_correct = []
# all_correct_valence = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     participants = []
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 0:
#             counter += 1
#         if row["charming"] >= 0:
#             counter += 1
#         if row["proud"]>= 0:
#             counter += 1
#         if row["safe"] >= 0:
#             counter += 1
#         if row["resolvable"] >= 0:
#             counter += 1
#         if row["prepared"] >= 0:
#             counter += 1
# #         if row["reach"] == 0:
# #             counter += 1
# #         if row["backed"] == 0:
# #             counter += 1
#         if row["shy"] <= 0:
#             counter += 1
#         if row["stutter"] <= 0:
#             counter += 1
#         if row["depriving"] <= 0:
#             counter += 1
#         if row["ungrateful"] <= 0:
#             counter += 1
#         if row["cruelness"] <= 0:
#             counter += 1
#         if row["repulse"] <= 0:
#             counter += 1
#         if row["slavery"] <= 0:
#             counter += 1
#         if row["terrorist"] <= 0:
#             counter += 1
# #         print(counter)
# #         if counter / 16 >= 0.8:
# #             counter_people += 1
    
#         correct_number = counter / 16
#         participants.append(correct_number)
        
#     all_correct_valence.append(sum(participants)/len(participants))
    
# print(all_correct_valence)
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
# #     how_many_people_correct.append(counter_people)
# #     eighty_correct.append(counter_people / len(dataframe))
    
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))

In [21]:
# # How many golden items did people get correct valence on average?

# how_many_people_correct = []
# all_correct_valence_participants = []
# participants = []

# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
# #     participants = []
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 0:
#             counter += 1
#         if row["charming"] >= 0:
#             counter += 1
#         if row["proud"]>= 0:
#             counter += 1
#         if row["safe"] >= 0:
#             counter += 1
#         if row["resolvable"] >= 0:
#             counter += 1
#         if row["prepared"] >= 0:
#             counter += 1
# #         if row["reach"] == 0:
# #             counter += 1
# #         if row["backed"] == 0:
# #             counter += 1
#         if row["shy"] <= 0:
#             counter += 1
#         if row["stutter"] <= 0:
#             counter += 1
#         if row["depriving"] <= 0:
#             counter += 1
#         if row["ungrateful"] <= 0:
#             counter += 1
#         if row["cruelness"] <= 0:
#             counter += 1
#         if row["repulse"] <= 0:
#             counter += 1
#         if row["slavery"] <= 0:
#             counter += 1
#         if row["terrorist"] <= 0:
#             counter += 1
# #         print(counter)
#         if counter / 16 == 1:
#             counter_people += 1
    
# #         correct_number = counter / 16
#     participants.append(counter_people)
# #         participants.append(correct_number)
        
# #     all_correct_valence_participants.append(sum(participants)/len(participants))
    
# print(participants)
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
# #     how_many_people_correct.append(counter_people)
# #     eighty_correct.append(counter_people / len(dataframe))
    
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))

In [22]:
# # How many of the 10 were they ideal? 

# how_many_people_correct = []
# love_c = []
# charming_c = []
# proud_c = []
# safe_c = []
# resolvable_c = []
# prepared_c = []
# reach_c = []
# backed_c = []
# shy_c = []
# stutter_c = []
# depriving_c = []
# ungrateful_c = []
# repulse_c = []
# cruelness_c = []
# slavery_c = []
# terrorist_c = []

# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
# #     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     # Scores for each item
#     love = 0
#     charming = 0
#     proud = 0
#     safe = 0
#     resolvable = 0
#     prepared = 0
#     reach = 0
#     backed = 0
#     shy = 0
#     stutter = 0
#     depriving = 0
#     ungrateful = 0
#     repulse = 0
#     cruelness = 0
#     slavery = 0
#     terrorist = 0
    
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] == 3:
#             love += 1
#         if row["charming"] == 3:
#             charming += 1
#         if row["proud"] == 2:
#             proud += 1
#         if row["safe"] == 2:
#             safe += 1
#         if row["resolvable"] == 1:
#             resolvable += 1
#         if row["prepared"] == 1:
#             prepared += 1
#         if row["reach"] == 0:
#             reach += 1
#         if row["backed"] == 0:
#             backed += 1
#         if row["shy"] == -1:
#             shy += 1
#         if row["stutter"] == -1:
#             stutter += 1
#         if row["depriving"] == -2:
#             depriving += 1
#         if row["ungrateful"] == -2:
#             ungrateful += 1
#         if row["cruelness"] == -3:
#             cruelness += 1
#         if row["repulse"] == -3:
#             repulse += 1
#         if row["slavery"] == -4:
#             slavery += 1
#         if row["terrorist"] == -4:
#             terrorist += 1
# #         print(counter)
# #         if counter / 16 >= 0.8:
# #             counter_people += 1
    
            
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
# #     how_many_people_correct.append(counter_people)
# #     print("love: ", love)
#     love_c.append(love / len(dataframe))
# #     print("charming: ", charming)
#     charming_c.append(charming / len(dataframe))
# #     print("proud: ", proud)
#     proud_c.append(proud / len(dataframe))
# #     print("safe: ", safe)
#     safe_c.append(safe / len(dataframe))
# #     print("resolvable: ", resolvable)
#     resolvable_c.append(resolvable / len(dataframe))
# #     print("prepared: ", prepared)
#     prepared_c.append(prepared / len(dataframe))
# #     print("reach: ", reach)
#     reach_c.append(reach / len(dataframe))
# #     print("backed: ", backed)
#     backed_c.append(backed / len(dataframe))
# #     print("shy: ", shy)
#     shy_c.append(shy / len(dataframe))
# #     print("stutter: ", stutter)
#     stutter_c.append(stutter / len(dataframe))
# #     print("depriving: ", depriving)
#     depriving_c.append(depriving / len(dataframe))
# #     print("ungrateful: ", ungrateful)
#     ungrateful_c.append(ungrateful / len(dataframe))
# #     print("cruelness: ", cruelness)
#     cruelness_c.append(cruelness / len(dataframe))
# #     print("repulse: ", repulse)
#     repulse_c.append(repulse / len(dataframe))
# #     print("slavery: ", slavery)
#     slavery_c.append(slavery / len(dataframe))
# #     print("terrorist: ", terrorist)
#     terrorist_c.append(terrorist / len(dataframe))
    
# print("love: ", sum(love_c) / len(love_c))
# print("charming: ", sum(charming_c) / len(charming_c))
# print("proud: ", sum(proud_c) / len(proud_c))
# print("safe: ", sum(safe_c) / len(safe_c))
# print("resolvable: ", sum(resolvable_c) / len(resolvable_c))
# print("prepared: ", sum(prepared_c) / len(prepared_c))
# print("reach: ", sum(reach_c) / len(reach_c))
# print("backed: ", sum(backed_c) / len(backed_c))
# print("shy: ", sum(shy_c) / len(shy_c))
# print("stutter: ", sum(stutter_c) / len(stutter_c))
# print("depriving: ", sum(depriving_c) / len(depriving_c))
# print("ungrateful: ", sum(ungrateful_c) / len(ungrateful_c))
# print("cruelness: ", sum(cruelness_c) / len(cruelness_c))
# print("repulse: ", sum(repulse_c) / len(repulse_c))
# print("slavery: ", sum(slavery_c) / len(slavery_c))
# print("terrorist: ", sum(terrorist_c) / len(terrorist_c))

    
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))


In [23]:
# # How many of the 10 were they within +/- 1? 

# how_many_people_correct = []
# love_c = []
# charming_c = []
# proud_c = []
# safe_c = []
# resolvable_c = []
# prepared_c = []
# reach_c = []
# backed_c = []
# shy_c = []
# stutter_c = []
# depriving_c = []
# ungrateful_c = []
# repulse_c = []
# cruelness_c = []
# slavery_c = []
# terrorist_c = []

# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
#     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     # Scores for each item
#     love = 0
#     charming = 0
#     proud = 0
#     safe = 0
#     resolvable = 0
#     prepared = 0
#     reach = 0
#     backed = 0
#     shy = 0
#     stutter = 0
#     depriving = 0
#     ungrateful = 0
#     repulse = 0
#     cruelness = 0
#     slavery = 0
#     terrorist = 0
    
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 2:
#             love += 1
#         if row["charming"] >= 2:
#             charming += 1
#         if row["proud"] >= 1 and row["proud"] <= 3:
#             proud += 1
#         if row["safe"] >= 1 and row["safe"] <= 3:
#             safe += 1
#         if row["resolvable"] >= 0 and row["resolvable"] <= 2:
#             resolvable += 1
#         if row["prepared"] >= 0 and row["prepared"] <= 2:
#             prepared += 1
#         if row["reach"] >= -1 and row["reach"] <= 1:
#             reach += 1
#         if row["backed"] >= -1 and row["backed"] <= 1:
#             backed += 1
#         if row["shy"] >= -2 and row["shy"] <= 0:
#             shy += 1
#         if row["stutter"] >= -2 and row["stutter"] <= 0:
#             stutter += 1
#         if row["depriving"] <= -1 and row["depriving"] >= -3:
#             depriving += 1
#         if row["ungrateful"] <= -1 and row["ungrateful"] >= -3:
#             ungrateful += 1
#         if row["cruelness"] <= -2:
#             cruelness += 1
#         if row["repulse"] <= -2:
#             repulse += 1
#         if row["slavery"] <= -3:
#             slavery += 1
#         if row["terrorist"] <= -3:
#             terrorist += 1
# #         print(counter)
# #         if counter / 16 >= 0.8:
# #             counter_people += 1
    
            
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
# #     how_many_people_correct.append(counter_people)
#     print("love: ", love)
#     love_c.append(love / len(dataframe))
#     print("charming: ", charming)
#     charming_c.append(charming / len(dataframe))
#     print("proud: ", proud)
#     proud_c.append(proud / len(dataframe))
#     print("safe: ", safe)
#     safe_c.append(safe / len(dataframe))
#     print("resolvable: ", resolvable)
#     resolvable_c.append(resolvable / len(dataframe))
#     print("prepared: ", prepared)
#     prepared_c.append(prepared / len(dataframe))
#     print("reach: ", reach)
#     reach_c.append(reach / len(dataframe))
#     print("backed: ", backed)
#     backed_c.append(backed / len(dataframe))
#     print("shy: ", shy)
#     shy_c.append(shy / len(dataframe))
#     print("stutter: ", stutter)
#     stutter_c.append(stutter / len(dataframe))
#     print("depriving: ", depriving)
#     depriving_c.append(depriving / len(dataframe))
#     print("ungrateful: ", ungrateful)
#     ungrateful_c.append(ungrateful / len(dataframe))
#     print("cruelness: ", cruelness)
#     cruelness_c.append(cruelness / len(dataframe))
#     print("repulse: ", repulse)
#     repulse_c.append(repulse / len(dataframe))
#     print("slavery: ", slavery)
#     slavery_c.append(slavery / len(dataframe))
#     print("terrorist: ", terrorist)
#     terrorist_c.append(terrorist / len(dataframe))
    
# print("love: ", sum(love_c) / len(love_c))
# print("charming: ", sum(charming_c) / len(charming_c))
# print("proud: ", sum(proud_c) / len(proud_c))
# print("safe: ", sum(safe_c) / len(safe_c))
# print("resolvable: ", sum(resolvable_c) / len(resolvable_c))
# print("prepared: ", sum(prepared_c) / len(prepared_c))
# print("reach: ", sum(reach_c) / len(reach_c))
# print("backed: ", sum(backed_c) / len(backed_c))
# print("shy: ", sum(shy_c) / len(shy_c))
# print("stutter: ", sum(stutter_c) / len(stutter_c))
# print("depriving: ", sum(depriving_c) / len(depriving_c))
# print("ungrateful: ", sum(ungrateful_c) / len(ungrateful_c))
# print("cruelness: ", sum(cruelness_c) / len(cruelness_c))
# print("repulse: ", sum(repulse_c) / len(repulse_c))
# print("slavery: ", sum(slavery_c) / len(slavery_c))
# print("terrorist: ", sum(terrorist_c) / len(terrorist_c))

    
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))


In [24]:
# # On average, how many of the 10 golden items did participants provide within +/- 2 rating for? 

# how_many_people_correct = []
# love_c = []
# charming_c = []
# proud_c = []
# safe_c = []
# resolvable_c = []
# prepared_c = []
# reach_c = []
# backed_c = []
# shy_c = []
# stutter_c = []
# depriving_c = []
# ungrateful_c = []
# repulse_c = []
# cruelness_c = []
# slavery_c = []
# terrorist_c = []

# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
#     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     # Scores for each item
#     love = 0
#     charming = 0
#     proud = 0
#     safe = 0
#     resolvable = 0
#     prepared = 0
#     reach = 0
#     backed = 0
#     shy = 0
#     stutter = 0
#     depriving = 0
#     ungrateful = 0
#     repulse = 0
#     cruelness = 0
#     slavery = 0
#     terrorist = 0
    
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 1:
#             love += 1
#         if row["charming"] >= 1:
#             charming += 1
#         if row["proud"] >= 0:
#             proud += 1
#         if row["safe"] >= 0:
#             safe += 1
#         if row["resolvable"] >= -1 and row["resolvable"] <= 3:
#             resolvable += 1
#         if row["prepared"] >= -1 and row["prepared"] <= 3:
#             prepared += 1
#         if row["reach"] >= -2 and row["reach"] <= 2:
#             reach += 1
#         if row["backed"] >= -2 and row["backed"] <= 2:
#             backed += 1
#         if row["shy"] >= -3 and row["shy"] <= 1:
#             shy += 1
#         if row["stutter"] >= -3 and row["stutter"] <= 1:
#             stutter += 1
#         if row["depriving"] <= 0:
#             depriving += 1
#         if row["ungrateful"] <= 0:
#             ungrateful += 1
#         if row["cruelness"] <= -1:
#             cruelness += 1
#         if row["repulse"] <= -1:
#             repulse += 1
#         if row["slavery"] <= -2:
#             slavery += 1
#         if row["terrorist"] <= -2:
#             terrorist += 1
# #         print(counter)
# #         if counter / 16 >= 0.8:
# #             counter_people += 1
    
            
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
# #     print("counter_people: ", counter_people)
# #     how_many_people_correct.append(counter_people)

#     print("love: ", love)
#     love_c.append(love / len(dataframe))
#     print("charming: ", charming)
#     charming_c.append(charming / len(dataframe))
#     print("proud: ", proud)
#     proud_c.append(proud / len(dataframe))
#     print("safe: ", safe)
#     safe_c.append(safe / len(dataframe))
#     print("resolvable: ", resolvable)
#     resolvable_c.append(resolvable / len(dataframe))
#     print("prepared: ", prepared)
#     prepared_c.append(prepared / len(dataframe))
#     print("reach: ", reach)
#     reach_c.append(reach / len(dataframe))
#     print("backed: ", backed)
#     backed_c.append(backed / len(dataframe))
#     print("shy: ", shy)
#     shy_c.append(shy / len(dataframe))
#     print("stutter: ", stutter)
#     stutter_c.append(stutter / len(dataframe))
#     print("depriving: ", depriving)
#     depriving_c.append(depriving / len(dataframe))
#     print("ungrateful: ", ungrateful)
#     ungrateful_c.append(ungrateful / len(dataframe))
#     print("cruelness: ", cruelness)
#     cruelness_c.append(cruelness / len(dataframe))
#     print("repulse: ", repulse)
#     repulse_c.append(repulse / len(dataframe))
#     print("slavery: ", slavery)
#     slavery_c.append(slavery / len(dataframe))
#     print("terrorist: ", terrorist)
#     terrorist_c.append(terrorist / len(dataframe))
    
# print("love: ", sum(love_c) / len(love_c))
# print("charming: ", sum(charming_c) / len(charming_c))
# print("proud: ", sum(proud_c) / len(proud_c))
# print("safe: ", sum(safe_c) / len(safe_c))
# print("resolvable: ", sum(resolvable_c) / len(resolvable_c))
# print("prepared: ", sum(prepared_c) / len(prepared_c))
# print("reach: ", sum(reach_c) / len(reach_c))
# print("backed: ", sum(backed_c) / len(backed_c))
# print("shy: ", sum(shy_c) / len(shy_c))
# print("stutter: ", sum(stutter_c) / len(stutter_c))
# print("depriving: ", sum(depriving_c) / len(depriving_c))
# print("ungrateful: ", sum(ungrateful_c) / len(ungrateful_c))
# print("cruelness: ", sum(cruelness_c) / len(cruelness_c))
# print("repulse: ", sum(repulse_c) / len(repulse_c))
# print("slavery: ", sum(slavery_c) / len(slavery_c))
# print("terrorist: ", sum(terrorist_c) / len(terrorist_c))


    
# # print(sum(how_many_people_correct) / len(how_many_people_correct))
# # print(min(how_many_people_correct))

In [25]:
# # If a golden item was -1, then count as “correct” if they were on the neutral or negative valence side regardless of 0, -1, -2, -3, or -4.
# # Same for positive.

# how_many_people_correct = []
# for key in dict_of_dfs.keys():
#     print("Change below")
#     dataframe = dict_of_dfs[key]
#     print(len(dataframe))
# # titanic[titanic["Age"] > 35]
#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
#     dataframe = dataframe[dataframe["love"] >= 0] # 3
#     dataframe = dataframe.loc[(dataframe["charming"] >= 0)] # 3
#     dataframe = dataframe.loc[(dataframe["proud"] >= 0)] # 2
#     dataframe = dataframe.loc[(dataframe["safe"] >= 0)] # 2
#     dataframe = dataframe.loc[(dataframe["resolvable"] >= 0)] # 1
#     dataframe = dataframe.loc[(dataframe["prepared"] >= 0)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
#     dataframe = dataframe.loc[(dataframe["shy"] <= 0)] # -1
#     dataframe = dataframe.loc[(dataframe["stutter"] <= 0)] # -1
#     dataframe = dataframe.loc[(dataframe["depriving"] <= 0)] # -2
#     dataframe = dataframe.loc[(dataframe["ungrateful"] <= 0)] # -2
#     dataframe = dataframe.loc[(dataframe["cruelness"] <= 0)] # -3
#     dataframe = dataframe.loc[(dataframe["repulse"] <= 0)] # -3
#     dataframe = dataframe.loc[(dataframe["slavery"] <= 0)] # -4
#     dataframe = dataframe.loc[(dataframe["terrorist"] <= 0)] # -4

    
# #     dict_of_dfs[key] = dataframe
#     print(len(dataframe))
#     how_many_people_correct.append(len(dataframe))
    
# print(sum(how_many_people_correct) / len(how_many_people_correct))
# print(min(how_many_people_correct))

In [26]:
# # Repeat as above but for 80%. 

# how_many_people_correct = []
# for key in dict_of_dfs.keys():
# #     print("Change below")
#     dataframe = dict_of_dfs[key]
#     print(len(dataframe))

#     dataframe = dataframe.loc[:,~dataframe.columns.duplicated()].copy() # Words may be duplicated...
    
#     counter_people = 0
#     for index, row in dataframe.iterrows(): # Row is the particpant
#         # Give a point if they get the ideal solution
#         counter = 0
#         if row["love"] >= 0:
#             counter += 1
#         if row["charming"] >= 0:
#             counter += 1
#         if row["proud"] >= 0:
#             counter += 1
#         if row["safe"] >= 0:
#             counter += 1
#         if row["resolvable"] >= 0:
#             counter += 1
#         if row["prepared"] >= 0:
#             counter += 1
# #         if row["reach"] == 0:
# #             counter += 1
# #         if row["backed"] == 0:
# #             counter += 1
#         if row["shy"] <= 0:
#             counter += 1
#         if row["stutter"] <= 0:
#             counter += 1
#         if row["depriving"] <= 0:
#             counter += 1
#         if row["ungrateful"] <= 0:
#             counter += 1
#         if row["cruelness"] <= 0:
#             counter += 1
#         if row["repulse"] <= 0:
#             counter += 1
#         if row["slavery"] <= 0:
#             counter += 1
#         if row["terrorist"] <= 0:
#             counter += 1
# #         print(counter)
#         if counter / 16 >= 0.8:
#             counter_people += 1
    
            
    
# #     dataframe = dataframe[dataframe["love"] >= 1] # 3
# #     dataframe = dataframe.loc[(dataframe["charming"] >= 1)] # 3
# #     dataframe = dataframe.loc[(dataframe["proud"] >= 0) & (dataframe["proud"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["safe"] >= 0) & (dataframe["safe"] <= 4)] # 2
# #     dataframe = dataframe.loc[(dataframe["resolvable"] >= -1) & (dataframe["resolvable"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["prepared"] >= -1) & (dataframe["prepared"] <= 3)] # 1
# #     dataframe = dataframe.loc[(dataframe["reach"] >= -2) & (dataframe["reach"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["backed"] >= -2) & (dataframe["backed"] <= 2)] # 0
# #     dataframe = dataframe.loc[(dataframe["shy"] >= -3) & (dataframe["shy"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["stutter"] >= -3) & (dataframe["stutter"] <= 1)] # -1
# #     dataframe = dataframe.loc[(dataframe["depriving"] >= -4) & (dataframe["depriving"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["ungrateful"] >= -4) & (dataframe["ungrateful"] <= 0)] # -2
# #     dataframe = dataframe.loc[(dataframe["cruelness"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["repulse"] <= -1)] # -3
# #     dataframe = dataframe.loc[(dataframe["slavery"] <= -2)] # -4
# #     dataframe = dataframe.loc[(dataframe["terrorist"] <= -2)] # -4

    
# #     dict_of_dfs[key] = dataframe
#     print("counter_people: ", counter_people)
#     how_many_people_correct.append(counter_people)
    
# print(sum(how_many_people_correct) / len(how_many_people_correct))
# print(min(how_many_people_correct))

In [27]:
# print(non_excluded) # # of non-excluded participants
# print(all_correct) # average % of golden items correct for participants who saw this list
# print(all_correct_1) # ave % of golden items within +/- 1 unit
# print(all_correct_2) # ave % of golden items within +/- 2 units
# print(all_correct_valence) # ave % of NON-neutral golden items in correct valence
# print(eighty_correct) # # of participants who got >80% of golden items correct
# print(eighty_correct_1) # # of participants who got >80% of golden items within +/- 1 unit
# print(eighty_correct_2) # # of participants who got > 80% of golden items within +/- 2 units
# print(participants) # # of participants who got NON-neutral golden items in correct valence

In [28]:
# sample_spread = pd.DataFrame()
# sample_spread['# of non-excluded participants'] = pd.Series(non_excluded)
# # sample_spread.assign("# of non-excluded participants"=non_excluded)
# sample_spread['average % of golden items correct for participants who saw this list'] = pd.Series(all_correct)
# sample_spread['ave % of golden items within +/- 1 unit'] = pd.Series(all_correct_1)
# sample_spread['ave % of golden items within +/- 2 units'] = pd.Series(all_correct_2)
# sample_spread['ave % of NON-neutral golden items in correct valence'] = pd.Series(all_correct_valence)
# sample_spread['# of participants who got >80% of golden items correct'] = pd.Series(eighty_correct)
# sample_spread['# of participants who got >80% of golden items within +/- 1 unit'] = pd.Series(eighty_correct_1)
# sample_spread['# of participants who got > 80% of golden items within +/- 2 units'] = pd.Series(eighty_correct_2)
# sample_spread['# of participants who got NON-neutral golden items in correct valence'] = pd.Series(participants)

# sample_spread
# sample_spread.to_csv('sample_spread.csv', encoding='utf-8')